# Wersja v15: Model Hybrydowy (Najlepsze Cechy + Stabilny Trening) nowa baza

## Część 1: Przygotowanie Danych i Zaawansowana Inżynieria Cech

In [1]:
# --- POCZĄTEK NOWEJ I KOMPLETNEJ CZĘŚCI 1 (WERSJA FINALNA) ---

import pandas as pd
import numpy as np
import os
import joblib
import gc
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from IPython.display import display

# --- Krok 1: Konfiguracja ---
ARTIFACTS_DIR = 'artifacts_v15_final'
PROCESSED_DATA_DIR = 'processed_data_v15_final'
MAX_TEXT_FEATURES = 20000
RANDOM_STATE = 42

os.makedirs(ARTIFACTS_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
print(f"Katalogi '{ARTIFACTS_DIR}' i '{PROCESSED_DATA_DIR}' są gotowe.")

# --- Krok 2: Wczytywanie Danych z Jawnie Wskazaną Kolumną 'locationPath' ---
def load_location_data(location_path='lokalizacja.csv'):
    df_loc = pd.read_csv(
        location_path, sep=',', header=None,
        names=['id', 'parent_id', 'name', 'type', 'full_name'], dtype=str
    )
    df_loc['id'] = pd.to_numeric(df_loc['id'], errors='coerce')
    df_loc['parent_id'] = pd.to_numeric(df_loc['parent_id'], errors='coerce')
    df_loc.dropna(subset=['id'], inplace=True)
    df_loc['id'] = df_loc['id'].astype('int64')
    return df_loc

def load_offers_data_final(offers_path='saleflats_2024_2025_v2.csv'):
    print(f"Wczytywanie pliku: {offers_path}...")
    try:
        # OSTATECZNA POPRAWKA: Jawnie definiujemy indeksy, w tym poprawny dla locationPath
        cols_map = {
            0: 'SaleId', 3: 'title', 4: 'description',
            5: 'area', 6: 'price', 52: 'locationPath' # Używamy potwierdzonego indeksu 52
        }
        
        df_selected = pd.read_csv(
            offers_path, 
            header=None, 
            sep=',', 
            quotechar='"',
            dtype=str,
            on_bad_lines='warn',
            usecols=list(cols_map.keys()), # Wczytujemy tylko te kolumny, które potrzebujemy
            low_memory=False
        )
        
        df_selected.columns = list(cols_map.values())
        print(f"Pomyślnie wczytano {len(df_selected)} wierszy.")
        return df_selected
    except Exception as e:
        print(f"Wystąpił KRYTYCZNY błąd: {e}")
        return pd.DataFrame()

df_loc = load_location_data()
df_offers = load_offers_data_final()
assert not df_offers.empty, "DataFrame z ofertami jest pusty."

id_to_name = dict(zip(df_loc['id'], df_loc['name']))
hierarchy_map = dict(zip(df_loc['id'], df_loc['parent_id']))
joblib.dump(id_to_name, os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))
joblib.dump(hierarchy_map, os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
print("Mapy pomocnicze zostały stworzone.")

# --- Krok 3: Inżynieria Cech (Logika z v14) ---
print("\nRozpoczynam inżynierię cech...")

df_offers.dropna(subset=['locationPath'], inplace=True)
df_offers = df_offers[~df_offers['locationPath'].str.contains('[a-zA-Z]', na=False)].copy()

path_cols = ['woj_id', 'pow_id', 'gmi_id', 'city_id', 'district_id', 'subdistrict_id', 'street_id']
path_df = df_offers['locationPath'].str.split(',', expand=True, n=len(path_cols)-1)
path_df.columns = path_cols[:path_df.shape[1]]
path_df = path_df.reindex(columns=path_cols, fill_value='0')
path_df = path_df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

df_offers['area'] = pd.to_numeric(df_offers['area'], errors='coerce')
df_offers['price'] = pd.to_numeric(df_offers['price'], errors='coerce')
df_full = pd.concat([df_offers.reset_index(drop=True), path_df.reset_index(drop=True)], axis=1)

valid_rows_conditions = (
    df_full['area'].notna() & df_full['price'].notna() & df_full['city_id'].notna() &
    (df_full['area'] > 0) & (df_full['price'] > 0) & (df_full['city_id'] != 0)
)
df_clean = df_full[valid_rows_conditions].copy()
df_clean['price_per_meter'] = df_clean['price'] / df_clean['area']
df_clean.reset_index(drop=True, inplace=True)
assert not df_clean.empty, "DataFrame jest pusty po filtrowaniu."

df_merged = pd.merge(df_clean, df_loc[['id', 'name']], left_on='district_id', right_on='id', how='left').rename(columns={'name': 'name_dist'})
df_merged = pd.merge(df_merged, df_loc[['id', 'name']], left_on='subdistrict_id', right_on='id', how='left').rename(columns={'name': 'name_subdist'})
df_merged['is_central_subdistrict'] = np.where(df_merged['name_dist'] == df_merged['name_subdist'], 1, 0)
final_cols = ['SaleId', 'title', 'description', 'area', 'price', 'price_per_meter', 'city_id', 'district_id', 'subdistrict_id', 'street_id', 'is_central_subdistrict']
df_processed = df_merged[final_cols].copy()
df_processed.rename(columns={'district_id': 'target_district_id', 'subdistrict_id': 'target_subdistrict_id', 'street_id': 'target_street_id'}, inplace=True)
df_processed['text_features'] = df_processed['title'].fillna('') + " " + df_processed['description'].fillna('')
df_processed.drop(columns=['title', 'description'], inplace=True)
df_processed.dropna(subset=['text_features'], inplace=True)
print(f"Zakończono inżynierię cech. Liczba wierszy w finalnym df_processed: {len(df_processed)}")

# --- Krok 4: Podział Danych i Transformacja ---
print("\nRozpoczynam podział danych i transformację...")
city_counts = df_processed['city_id'].value_counts()
valid_cities = city_counts[city_counts > 1].index
df_filtered = df_processed[df_processed['city_id'].isin(valid_cities)].copy()
assert not df_filtered.empty, "Po odfiltrowaniu miast z 1 ofertą ramka danych jest pusta."
target_columns = ['target_district_id', 'target_subdistrict_id', 'target_street_id']
X = df_filtered.drop(columns=target_columns)
y = df_filtered[target_columns]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=X['city_id'])
print(f"Podział danych: {len(X_train)} próbek treningowych, {len(X_val)} próbek walidacyjnych.")
numeric_features = ['area', 'price', 'price_per_meter', 'is_central_subdistrict']
imputer = SimpleImputer(strategy='median'); scaler = StandardScaler()
X_train_num = scaler.fit_transform(imputer.fit_transform(X_train[numeric_features]))
X_val_num = scaler.transform(imputer.transform(X_val[numeric_features]))
joblib.dump(imputer, os.path.join(ARTIFACTS_DIR, 'imputer.joblib'))
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))
vectorizer = TfidfVectorizer(max_features=MAX_TEXT_FEATURES, ngram_range=(1, 2))
X_train_text = vectorizer.fit_transform(X_train['text_features'])
X_val_text = vectorizer.transform(X_val['text_features'])
joblib.dump(vectorizer, os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))
X_train_city = X_train['city_id'].values
X_val_city = X_val['city_id'].values
X_val_sale_ids = X_val['SaleId'].values
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_sale_ids.npy'), X_val_sale_ids)
y_train.to_pickle(os.path.join(PROCESSED_DATA_DIR, 'y_train.pkl'))
y_val.to_pickle(os.path.join(PROCESSED_DATA_DIR, 'y_val.pkl'))
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'), X_train_text)
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'), X_val_text)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'), X_train_num)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'), X_val_num)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'), X_train_city)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'), X_val_city)
print("Transformatory dopasowane i wszystkie dane zapisane.")
del df_loc, df_offers, df_processed, X, y, X_train, X_val, y_train, y_val, df_filtered, df_merged, df_clean, df_full
del X_train_text, X_val_text, X_train_num, X_val_num, X_train_city, X_val_city
gc.collect()
print("\n--- CAŁA CZĘŚĆ 1 ZAKOŃCZONA ---")

Katalogi 'artifacts_v15_final' i 'processed_data_v15_final' są gotowe.
Wczytywanie pliku: saleflats_2024_2025_v2.csv...
Pomyślnie wczytano 1467263 wierszy.
Mapy pomocnicze zostały stworzone.

Rozpoczynam inżynierię cech...
Zakończono inżynierię cech. Liczba wierszy w finalnym df_processed: 1250259

Rozpoczynam podział danych i transformację...
Podział danych: 997916 próbek treningowych, 249479 próbek walidacyjnych.
Transformatory dopasowane i wszystkie dane zapisane.

--- CAŁA CZĘŚĆ 1 ZAKOŃCZONA ---


# Część 2: Budowa, Trening i Inferencia Modelu

In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import scipy.sparse
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

ARTIFACTS_DIR = 'artifacts_v14'
PROCESSED_DATA_DIR = 'processed_data_v14'
RANDOM_STATE = 42
MODEL_PATH = os.path.join(ARTIFACTS_DIR, 'best_location_model_v14.keras')

In [2]:
print("Wczytywanie artefaktów...")
hierarchy_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
id_to_name = joblib.load(os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))
print("Wczytywanie przetworzonych danych...")
X_train_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'))
X_val_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'))
X_train_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'))
X_val_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'))
X_train_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'))
X_val_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'))
X_val_sale_ids = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_sale_ids.npy'))
y_train = pd.read_pickle(os.path.join(PROCESSED_DATA_DIR, 'y_train.pkl'))
y_val = pd.read_pickle(os.path.join(PROCESSED_DATA_DIR, 'y_val.pkl'))
print("Dane wczytane.")

Wczytywanie artefaktów...
Wczytywanie przetworzonych danych...
Dane wczytane.


In [3]:
def create_mapper(series_list):
    unique_ids = pd.unique(pd.concat(series_list))
    return {val: i for i, val in enumerate(unique_ids)}

def map_labels(series, mapper):
    return series.map(mapper).fillna(0).astype(int).values

district_map = create_mapper([y_train['target_district_id'], y_val['target_district_id']])
subdistrict_map = create_mapper([y_train['target_subdistrict_id'], y_val['target_subdistrict_id']])
street_map = create_mapper([y_train['target_street_id'], y_val['target_street_id']])
city_map = create_mapper([pd.Series(X_train_city), pd.Series(X_val_city)])

joblib.dump(district_map, os.path.join(ARTIFACTS_DIR, 'district_map.joblib'))
joblib.dump(subdistrict_map, os.path.join(ARTIFACTS_DIR, 'subdistrict_map.joblib'))
joblib.dump(street_map, os.path.join(ARTIFACTS_DIR, 'street_map.joblib'))
joblib.dump(city_map, os.path.join(ARTIFACTS_DIR, 'city_map.joblib'))

y_district_train_mapped = map_labels(y_train['target_district_id'], district_map)
y_subdistrict_train_mapped = map_labels(y_train['target_subdistrict_id'], subdistrict_map)
y_street_train_mapped = map_labels(y_train['target_street_id'], street_map)
y_district_val_mapped = map_labels(y_val['target_district_id'], district_map)
y_subdistrict_val_mapped = map_labels(y_val['target_subdistrict_id'], subdistrict_map)
y_street_val_mapped = map_labels(y_val['target_street_id'], street_map)
X_train_city_mapped = map_labels(pd.Series(X_train_city), city_map)
X_val_city_mapped = map_labels(pd.Series(X_val_city), city_map)

MAX_TEXT_FEATURES = 20000
NUM_DISTRICTS, NUM_SUBDISTRICTS, NUM_STREETS, NUM_CITIES = len(district_map), len(subdistrict_map), len(street_map), len(city_map)
NUM_FEATURES = X_train_num.shape[1]

input_text = Input(shape=(MAX_TEXT_FEATURES,), name='text_input', sparse=True)
input_num = Input(shape=(NUM_FEATURES,), name='num_input')
input_city = Input(shape=(1,), name='city_input')

text_branch = Dense(128, activation='relu')(input_text); text_branch = Dropout(0.3)(text_branch)
num_branch = Dense(64, activation='relu')(input_num); num_branch = Dense(32, activation='relu')(num_branch)
city_branch = Embedding(input_dim=NUM_CITIES, output_dim=50, name='city_embedding')(input_city); city_branch = Flatten()(city_branch)
combined = Concatenate()([text_branch, num_branch, city_branch])
z = Dense(512, activation='relu')(combined); z = Dropout(0.5)(z)
z = Dense(256, activation='relu')(z); z = Dropout(0.5)(z)
z = Dense(128, activation='relu')(z); z = Dropout(0.5)(z)

output_district = Dense(NUM_DISTRICTS, activation='softmax', name='district_output')(z)
output_subdistrict = Dense(NUM_SUBDISTRICTS, activation='softmax', name='subdistrict_output')(z)
output_street = Dense(NUM_STREETS, activation='softmax', name='street_output')(z)

model = Model(inputs=[input_text, input_num, input_city], outputs=[output_district, output_subdistrict, output_street])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 20000)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ num_input (InputLayer)        │ (None, 4)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │       2,560,128 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │             320 │ num_input[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_embedding (Embedding)    │ (None, 1, 50)             │         276,800 │ city_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │           2,080 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 50)                │               0 │ city_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 210)               │               0 │ dropout[0][0],             │
│                               │                           │                 │ dense_2[0][0],             │
│                               │                           │                 │ flatten[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 512)               │         108,032 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 512)               │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 256)               │         131,328 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 256)               │               0 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_5 (Dense)               │ (None, 128)               │          32,896 │ dropout_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_3 (Dropout)           │ (None, 128)               │               0 │ dense_5[0][0]              │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 7,730,945 (29.49 MB)

 Trainable params: 7,730,945 (29.49 MB)

 Non-trainable params: 0 (0.00 B)

## Trening z Prostą Stratą i Umiarkowanymi Wagami

In [4]:
district_loss = 'sparse_categorical_crossentropy'
subdistrict_loss = 'sparse_categorical_crossentropy'
street_loss = 'sparse_categorical_crossentropy'

model.compile(
    optimizer='adam',
    loss={'district_output': district_loss, 'subdistrict_output': subdistrict_loss, 'street_output': street_loss},
    loss_weights={'district_output': 1.0, 'subdistrict_output': 1.2, 'street_output': 1.5},
    metrics={'district_output': 'accuracy', 'subdistrict_output': 'accuracy', 'street_output': 'accuracy'}
)

subdistrict_train_weights = np.where(y_train['target_subdistrict_id'].values > 0, 1.5, 1.0)
street_train_weights = np.where(y_train['target_street_id'].values > 0, 2.0, 1.0)
sample_weights_list = [np.ones(len(y_train)), subdistrict_train_weights, street_train_weights]

X_train_dict = {'text_input': X_train_text, 'num_input': X_train_num, 'city_input': X_train_city_mapped}
y_train_list = [y_district_train_mapped, y_subdistrict_train_mapped, y_street_train_mapped]
X_val_dict = {'text_input': X_val_text, 'num_input': X_val_num, 'city_input': X_val_city_mapped}
y_val_list = [y_district_val_mapped, y_subdistrict_val_mapped, y_street_val_mapped]

callbacks = [
    ModelCheckpoint(MODEL_PATH, monitor='val_loss', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
]

history = model.fit(
    X_train_dict, y_train_list,
    sample_weight=sample_weights_list,
    validation_data=(X_val_dict, y_val_list),
    epochs=20, batch_size=128,
    callbacks=callbacks
)

Epoch 1/20
6934/6935 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - district_output_accuracy: 0.7086 - district_output_loss: 1.0084 - loss: 17.3585 - street_output_accuracy: 0.6347 - street_output_loss: 7.4994 - subdistrict_output_accuracy: 0.5137 - subdistrict_output_loss: 4.2508
Epoch 1: val_loss improved from inf to 7.58718, saving model to artifacts_v14\best_location_model_v14.keras
6935/6935 ━━━━━━━━━━━━━━━━━━━━ 509s 73ms/step - district_output_accuracy: 0.7086 - district_output_loss: 1.0083 - loss: 17.3579 - street_output_accuracy: 0.6347 - street_output_loss: 7.4992 - subdistrict_output_accuracy: 0.5137 - subdistrict_output_loss: 4.2507 - val_district_output_accuracy: 0.7705 - val_district_output_loss: 0.6372 - val_loss: 7.5872 - val_street_output_accuracy: 0.6424 - val_street_output_loss: 3.0768 - val_subdistrict_output_accuracy: 0.5622 - val_subdistrict_output_loss: 1.9459
Epoch 2/20
6934/6935 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - district_output_accuracy: 0.7698 - district_output_loss: 0.

## Predykcja Top-K (k=5) i Analiza Błędów

In [5]:
print(f"Wczytywanie najlepszego modelu z: {MODEL_PATH}")
best_model = tf.keras.models.load_model(MODEL_PATH)
print("Model wczytany pomyślnie.")

inv_district_map = {v: k for k, v in district_map.items()}
inv_subdistrict_map = {v: k for k, v in subdistrict_map.items()}
inv_street_map = {v: k for k, v in street_map.items()}

def find_best_consistent_path(top_k_districts, top_k_subdistricts, top_k_streets, hierarchy_map):
    for dist_id in top_k_districts:
        for sub_id in top_k_subdistricts:
            if sub_id != 0 and hierarchy_map.get(sub_id) != dist_id:
                continue
            for street_id in top_k_streets:
                if street_id != 0:
                    expected_parent = sub_id if sub_id != 0 else dist_id
                    if hierarchy_map.get(street_id) != expected_parent:
                        continue
                return dist_id, sub_id, street_id
    return top_k_districts[0], top_k_subdistricts[0], top_k_streets[0]

num_samples = 1000
k = 5
indices = np.random.choice(range(len(y_val)), num_samples, replace=False)
results_data = []

print(f"Przeprowadzanie predykcji Top-{k} z post-processingiem na {num_samples} próbkach...")
for i in indices:
    dense_vector_sample = X_val_text[i].toarray()
    current_sale_id = X_val_sale_ids[i]
    input_sample = {
        'text_input': dense_vector_sample,
        'num_input': np.expand_dims(X_val_num[i], axis=0),
        'city_input': np.expand_dims(map_labels(pd.Series(X_val_city[i]), city_map), axis=0)
    }
    
    preds = best_model.predict(input_sample, verbose=0)
    pred_district_probs, pred_subdistrict_probs, pred_street_probs = preds[0][0], preds[1][0], preds[2][0]

    top_k_district_ids = [inv_district_map.get(idx, 0) for idx in np.argsort(pred_district_probs)[::-1][:k]]
    top_k_subdistrict_ids = [inv_subdistrict_map.get(idx, 0) for idx in np.argsort(pred_subdistrict_probs)[::-1][:k]]
    top_k_street_ids = [inv_street_map.get(idx, 0) for idx in np.argsort(pred_street_probs)[::-1][:k]]

    final_district, final_subdistrict, final_street = find_best_consistent_path(
        top_k_district_ids, top_k_subdistrict_ids, top_k_street_ids, hierarchy_map
    )
    
    true_vals = y_val.iloc[i]
    results_data.append({
        'SaleId': current_sale_id,
        'City_ID': X_val_city[i],
        'True_District': true_vals['target_district_id'], 'Pred_District': final_district,
        'True_SubDistrict': true_vals['target_subdistrict_id'], 'Pred_SubDistrict': final_subdistrict,
        'True_Street': true_vals['target_street_id'], 'Pred_Street': final_street
    })

df_results = pd.DataFrame(results_data)

df_results['Correct_District'] = (df_results['True_District'] == df_results['Pred_District'])
df_results['Correct_SubDistrict'] = (df_results['True_SubDistrict'] == df_results['Pred_SubDistrict'])
df_results['Correct_Street'] = (df_results['True_Street'] == df_results['Pred_Street'])
df_results['Correct_Overall'] = df_results['Correct_District'] & df_results['Correct_SubDistrict'] & df_results['Correct_Street']

acc_district = df_results['Correct_District'].mean()
acc_subdistrict = df_results['Correct_SubDistrict'].mean()
acc_street = df_results['Correct_Street'].mean()
acc_overall = df_results['Correct_Overall'].mean()

print("\n--- Wyniki Ewaluacji (po Top-K i Post-processingu) ---")
print(f"Dokładność dla Dzielnic: {acc_district:.2%}")
print(f"Dokładność dla Pod-dzielnic: {acc_subdistrict:.2%}")
print(f"Dokładność dla Ulic: {acc_street:.2%}")
print(f"\nDokładność CAŁKOWITA (wszystkie poziomy poprawne): {acc_overall:.2%}")

Wczytywanie najlepszego modelu z: artifacts_v14\best_location_model_v14.keras
Model wczytany pomyślnie.
Przeprowadzanie predykcji Top-5 z post-processingiem na 1000 próbkach...

--- Wyniki Ewaluacji (po Top-K i Post-processingu) ---
Dokładność dla Dzielnic: 89.20%
Dokładność dla Pod-dzielnic: 62.60%
Dokładność dla Ulic: 66.80%

Dokładność CAŁKOWITA (wszystkie poziomy poprawne): 46.20%


In [6]:
for col in ['City_ID', 'True_District', 'Pred_District', 'True_SubDistrict', 'Pred_SubDistrict', 'True_Street', 'Pred_Street']:
    df_results[f'{col}_Name'] = df_results[col].apply(lambda x: id_to_name.get(x, 'Brak'))

df_errors = df_results[~df_results['Correct_Overall']].copy()
print("\n--- Analiza Błędów ---")

if not df_errors.empty:
    district_errors = df_errors[~df_errors['Correct_District']]
    if not district_errors.empty:
        print("\nTop 10 najczęstszych pomyłek na poziomie DZIELNICY:")
        display(district_errors.groupby(['True_District_Name', 'Pred_District_Name']).size().nlargest(10))
    else: print("\nBrak błędów na poziomie DZIELNICY!")
        
    subdistrict_errors = df_errors[df_errors['Correct_District'] & ~df_errors['Correct_SubDistrict']]
    if not subdistrict_errors.empty:
        print("\nTop 10 najczęstszych pomyłek na poziomie POD-DZIELNICY (przy poprawnej dzielnicy):")
        display(subdistrict_errors.groupby(['True_SubDistrict_Name', 'Pred_SubDistrict_Name']).size().nlargest(10))
    else: print("\nBrak błędów na poziomie POD-DZIELNICY przy poprawnych dzielnicach.")
else: print("\nModel nie popełnił żadnych błędów!")

def build_path(row, prefix):
    city = row['City_ID_Name'] if row['City_ID_Name'] != 'Brak' else '?'
    district = row[f'{prefix}_District_Name'] if row[f'{prefix}_District_Name'] != 'Brak' else '?'
    subdistrict = row[f'{prefix}_SubDistrict_Name'] if row[f'{prefix}_SubDistrict_Name'] != 'Brak' else '?'
    street = row[f'{prefix}_Street_Name'] if row[f'{prefix}_Street_Name'] != 'Brak' else '?'
    return f"{city} -> {district} -> {subdistrict} -> {street}"

df_results['True_Loc'] = df_results.apply(lambda row: build_path(row, 'True'), axis=1)
df_results['Predict_Loc'] = df_results.apply(lambda row: build_path(row, 'Pred'), axis=1)

print("\n--- Losowe wyniki predykcji ---")
df_display = df_results[['SaleId', 'True_Loc', 'Predict_Loc', 'Correct_Overall']].set_index('SaleId')
display(df_display.sample(min(20, len(df_display)), random_state=RANDOM_STATE))


--- Analiza Błędów ---

Top 10 najczęstszych pomyłek na poziomie DZIELNICY:


True_District_Name    Pred_District_Name  
Bielany               Praga-południe          6
Brak                  Wrocław-krzyki          6
                      Kraków-krowodrza        4
                      Mokotów                 4
                      Ursynów                 3
                      Wrocław-stare miasto    3
Kraków-podgórze       Kraków-krowodrza        3
Wawer                 Białołęka               3
Wrocław-stare miasto  Wrocław-krzyki          3
Łódź-bałuty           Łódź-górna              3
dtype: int64


Top 10 najczęstszych pomyłek na poziomie POD-DZIELNICY (przy poprawnej dzielnicy):


True_SubDistrict_Name   Pred_SubDistrict_Name
Śródmieście             Brak                     18
Centrum                 Brak                     10
Sielec                  Brak                      5
Szwederowo              Brak                      5
Jasień                  Brak                      4
Bródno                  Brak                      3
Bydgoskie przedmieście  Brak                      3
Czyste                  Brak                      3
Mikulczyce              Brak                      3
Nowe miasto             Brak                      3
dtype: int64


--- Losowe wyniki predykcji ---


,True_Loc,Predict_Loc,Correct_Overall
SaleId,,,
3492916,Debrzno -> ? -> ? -> ?,Debrzno -> ? -> ? -> ?,True
2827362,Wrocław -> Wrocław-fabryczna -> Żerniki -> Gra...,Wrocław -> Wrocław-fabryczna -> Fabryczna -> ?,False
2441083,Koszalin -> ? -> ? -> ?,Koszalin -> ? -> ? -> ?,True
4275257,Rzeszów -> ? -> Dworzysko -> ?,Rzeszów -> ? -> ? -> ?,False
3741723,Legionowo -> ? -> ? -> ?,Legionowo -> ? -> ? -> ?,True
602807,Kowale -> ? -> ? -> ?,Kowale -> ? -> ? -> ?,True
2385401,Warszawa -> Praga-południe -> Grochów -> ?,Warszawa -> Bielany -> ? -> ?,False
294324,Gorzów wielkopolski -> ? -> Śródmieście -> Edw...,Gorzów wielkopolski -> ? -> ? -> ?,False
579476,Nakło nad notecią -> ? -> ? -> ?,Nakło nad notecią -> ? -> ? -> ?,True
